# [I.02] Profitability Analysis

## Theory and Profitability Formulas

See handout **I.02** on Canvas.

## Example (Seider 17.14 and 17.29)

A process, projected to have a total depreciable capital of `$90 MM`, with no allocated costs for outside utilities, is to be installed over a 3-year period (2018–2020) and expected to operate for 15 yr. Just prior to startup, `$40 MM` of working capital is required. At 90% production capacity, (projected to be achieved by year 3 of operation), sales revenues, S, are projected to be `$150 MM/yr` and the total annual production cost, excluding depreciation, is projected to be `$100 MM/yr`. The plant is projected to operate at 50% of 90% (i.e., 45%) and 75% of 90% (i.e., 67.5%), respectively, in years 1 and 2 of operation. During these years, $S$ is `$75 MM` and `$113 MM`, respectively, and $C$ is `$55 MM` and `$78 MM`, respectively.

### A. Simple Profitability Metrics

Take straight-line depreciation at 8%. Compute the venture profit for an interest rate of 20%, the return on investment (ROI) and the payback period (PBP).

In [1]:
C_TDC = 90e+6  # $, total capital investment
C_WC = 40e+6  # $, total working capital, needed for ongoing costs, e.g., salaries
C_TCI = C_TDC + C_WC  # $, total capital investment

# Since production capacity is variable over the plant life,
# we will calculate for Year 3, when the capacity has maxed out

S = 150e+6  # $/yr, sales
D = 0.08 * C_TDC  # $/yr, depreciation = 8% of total depreciable capital, C_TDC
C = 100e+6  # $/yr, costs

t = 0.21 + 0.0825  # /yr, tax rate, federal 21% + MD 8.25%

r = 0.20  # /yr, interest rate for venture

In [2]:
VP = (1 - t) * (S - C - D) - r * C_TCI  # $/yr, venture profit
print(f"Venture profit = {VP} $/yr")

Venture profit = 4281000.0 $/yr


In [3]:
PBP = C_TDC / ((1 - t) * (S - C - D) + D)  # yr, payback period
print(f"Payback period = {PBP:.2f} yr")

Payback period = 2.40 yr


### B. Rigorous Profitability Metrics: NPV and IRR

Using MACRS depreciation, calculate (i) the net present value (NPV) for a nominal interest rate of return of 15% compounded annually, and (ii) the investor's rate of return (IRR), which is the nominal interest rate corresponding to NPV = 0.

In [4]:
import numpy as np
import pandas as pd

calendar_year = np.linspace(2026, 2040, 15).astype(int)
year = (calendar_year - 2025).astype(int)

In [5]:
df = pd.DataFrame({'Year': year}).set_index(calendar_year)

df['C_TDC'] = np.where(df['Year'] <= 3, -30, 0)
df['C_WC'] = np.where(df['Year'] == 3, -40, np.where(df['Year'] == 15, 40, 0))

In [6]:
df['MACRS'] = [0, 0, 0, 20, 32, 19.2, 11.52, 11.52, 5.76, 0, 0, 0, 0, 0, 0]
df['Depreciation'] = df['MACRS'] / 100 * C_TDC / 1e6

In [7]:
df['Costs'] = np.select([df['Year'] <= 3, df['Year'] == 4, df['Year'] == 5], [0, 55, 78], default=100)
df['Sales'] = np.select([df['Year'] <= 3, df['Year'] == 4, df['Year'] == 5], [0, 75, 113], default=150)
df['Earnings'] = (1 - t) * (df['Sales'] - df['Costs'] - df['Depreciation'])

In [8]:
df['Cash Flow'] = df['C_TDC'] + df['C_WC'] + df['Depreciation'] + df['Earnings']

In [9]:
i = 0.15  # nominal interest rate

df['Discounted Cash Flow'] = df['Cash Flow'] / (1 + i)**(df['Year'] - 1)
df['Cumulative Present Value'] = df['Discounted Cash Flow'].cumsum()

In [10]:
df.style.format("{:.2f}")

,Year,C_TDC,C_WC,MACRS,Depreciation,Costs,Sales,Earnings,Cash Flow,Discounted Cash Flow,Cumulative Present Value
2026,1.00,-30.00,0.00,0.00,0.00,0.00,0.00,0.00,-30.00,-30.00,-30.00
2027,2.00,-30.00,0.00,0.00,0.00,0.00,0.00,0.00,-30.00,-26.09,-56.09
2028,3.00,-30.00,-40.00,0.00,0.00,0.00,0.00,0.00,-70.00,-52.93,-109.02
2029,4.00,0.00,0.00,20.00,18.00,55.00,75.00,1.42,19.41,12.77,-96.25
2030,5.00,0.00,0.00,32.00,28.80,78.00,113.00,4.39,33.19,18.97,-77.28
2031,6.00,0.00,0.00,19.20,17.28,100.00,150.00,23.15,40.43,20.10,-57.18
2032,7.00,0.00,0.00,11.52,10.37,100.00,150.00,28.04,38.41,16.60,-40.57
2033,8.00,0.00,0.00,11.52,10.37,100.00,150.00,28.04,38.41,14.44,-26.13
2034,9.00,0.00,0.00,5.76,5.18,100.00,150.00,31.71,36.89,12.06,-14.07
2035,10.00,0.00,0.00,0.00,0.00,100.00,150.00,35.38,35.38,10.06,-4.02


In [11]:
df.loc[2040].loc['Cumulative Present Value']

np.float64(35.34455852225903)

In [12]:
def npv(i):
    df = pd.DataFrame({'Year': year}).set_index(calendar_year)

    df['C_TDC'] = np.where(df['Year'] <= 3, -30, 0)
    df['C_WC'] = np.where(df['Year'] == 3, -40, np.where(df['Year'] == 15, 40, 0))
    df['MACRS'] = [0, 0, 0, 20, 32, 19.2, 11.52, 11.52, 5.76, 0, 0, 0, 0, 0, 0]
    df['Depreciation'] = df['MACRS'] / 100 * C_TDC / 1e6
    df['Costs'] = np.select([df['Year'] <= 3, df['Year'] == 4, df['Year'] == 5], [0, 55, 78], default=100)
    df['Sales'] = np.select([df['Year'] <= 3, df['Year'] == 4, df['Year'] == 5], [0, 75, 113], default=150)
    df['Earnings'] = (1 - t) * (df['Sales'] - df['Costs'] - df['Depreciation'])
    df['Cash Flow'] = df['C_TDC'] + df['C_WC'] + df['Depreciation'] + df['Earnings']
    df['Discounted Cash Flow'] = df['Cash Flow'] / (1 + i)**(df['Year'] - 1)
    df['Cumulative Present Value'] = df['Discounted Cash Flow'].cumsum()
    return df.loc[2040].loc['Cumulative Present Value']

In [13]:
npv(0.15)

np.float64(35.34455852225903)

In [14]:
import scipy as sci

res = sci.optimize.root_scalar(npv, x0=0.15)

In [15]:
print(res)
print()
print(f"IRR = {res.root * 100:.2f}%")

      converged: True
           flag: converged
 function_calls: 10
     iterations: 5
           root: 0.20495876139629482
         method: newton

IRR = 20.50%
